In [ ]:
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.stem import PorterStemmer
import re
import tensorflow
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from sklearn.metrics import classification_report,accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from imblearn.over_sampling import RandomOverSampler

In [7]:
pip install tensorflow

  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached tensorflow_estimator-2.9.0-py2.py3-none-any.whl (438 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached libclang-14.0.1-py2.py3-none-win_amd64.whl (14.2 MB)
  Using cached keras-2.9.0-py2.py3-none-any.whl (1.6 MB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached tensorboard-2.9.1-py3-none-any.whl (5.8 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.2.0-py3-none-any.whl (151 kB)
 

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
## Test Data
fake_test_data = pd.read_csv('./Data/test.csv')

In [ ]:
fake_test_data.head()

In [ ]:
## create dataset
fake_news_df = pd.read_csv('./Data/train.csv')

In [ ]:
fake_news_df.head()

In [ ]:
## Merge data for new column

fake_news_df["merged"] = fake_news_df["title"]+" "+fake_news_df["author"]

fake_news_df.head()

In [ ]:
#  check null values
fake_news_df.isnull().sum()

In [ ]:
# drop null values
fake_news_df=fake_news_df.dropna()

In [ ]:
# Now count the Unique values using np.unique to find the elements of an array as parameter
count = np.unique(fake_news_df['label'], return_counts=True)
print(count)

In [ ]:
import seaborn as sns
sns.countplot(x='label', data = fake_news_df)

In [ ]:
# Dependent Features
Y = fake_news_df['label']

# Independent Features
X = fake_news_df.drop('label', axis=1)

In [ ]:
# change the dataset into numerical representation
# make the copy of Dependent Featues and reset the index because we drop the nan values due to this 
# the index is disturb
messages = X.copy()
# messages.reset_index(inplace=True)

In [ ]:
messages['title'][1]

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
# Data processing
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))

In [ ]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['merged'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus[1]

In [ ]:
voc_size = 5000
onehot_repre = [one_hot(words, voc_size) for words in corpus]
onehot_repre[0]

In [ ]:
# Word embedding
sent_length = 20
embedded_docs = pad_sequences(onehot_repre, padding='pre', maxlen=sent_length)
print(embedded_docs[0])

In [ ]:
## CREATING A MODEL

In [ ]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
x_final = np.array(embedded_docs)
y_final = np.array(Y)

In [ ]:
x_final.shape, y_final.shape

In [ ]:
## SPLIT THE DATA INTO TRAIN AND TEST
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.1, random_state=42)

In [ ]:
# MODEL TRAINING
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=64)


In [ ]:
# PERFORMANCE METRICS AND ACCURACY
classes_x = (model.predict(X_test) > 0.5).astype("int32")

In [ ]:
# Confusion Matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, classes_x)

In [ ]:
#Accuracy Score
from sklearn.metrics import accuracy_score
score_0 = accuracy_score(y_test,classes_x)
print(score_0)

In [2]:
model_1 = LogisticRegression(max_iter=900)
model_1.fit_resample(X_train,y_train)
pred_1 = model_1.predict(X_test)
# cr1    = classification_report(y_test,pred_1)
# print(cr1)
score_1 = accuracy_score(y_test,pred_1)
print(score_1)

NameError: name 'LogisticRegression' is not defined

In [3]:
# Resample the training data with the RandomOversampler
#from imblearn.over_sampling import RandomOverSampler

#OS = RandomOverSampler(sampling_strategy='auto', random_state=0)
#osx, osy = OS.fit_sample(X, y)

model_3 = RandomOverSampler(random_state=1)
x_sam,y_sam =model_3.fit_resample(X_train,y_train)
pred_3 = model_3.predict(X_test)
# cr3    = classification_report(y_test,pred_3)
# print(cr3)
score_3 = accuracy_score(y_test,pred_3)
print(score_3)

NameError: name 'RandomOverSampler' is not defined

In [4]:
model_4 = RandomForestClassifier()
model_4.fit(X_train,y_train)
pred_4 = model_4.predict(X_test)
# cr4    = classification_report(y_test,pred_4)
# print(cr4)
score_4 = accuracy_score(y_test,pred_4)
print(score_4)

NameError: name 'RandomForestClassifier' is not defined

In [5]:
from imblearn.under_sampling import ClusterCentroids

model_5 = ClusterCentroids()
model_5.fit(X_train,y_train)
pred_5 = model_5.predict(X_test)
# cr5    = classification_report(y_test,pred_5)
# print(cr5)
score_5 = accuracy_score(y_test,pred_5)
print(score_5)

NameError: name 'X_train' is not defined

In [ ]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN

model_6 = SMOTEENN()
model_6.fit(X_train,y_train)
pred_6 = model_6.predict(X_test)
# cr6    = classification_report(y_test,pred_6)
# print(cr6)
score_6 = accuracy_score(y_test,pred_6)